In [ ]:
!pip install OpenDartReader

In [3]:
from financial_data_making import financial_dict
import OpenDartReader
import pandas as pd

# df = pd.read_csv('./merged_code.csv')
call =  financial_dict('4fb4acc1d37b1dae74040e64a9674e018ccfd7fe') # api_key

df = pd.read_csv('./opendart_stockcode_finance.csv', dtype={'code': str})
df1 = df[df['y'] != '0']
print(df.shape)
print(df1.shape)
df1.head()
# call.dart_extract('6840', 2020)
# call.html_extract('027410', 2021)

(1049, 3)
(707, 3)


,code,y,prev_y
0,095570,"1,481,445,371,745","1,355,044,449,955"
1,006840,"5,004,334,961,394","4,548,736,656,905"
2,027410,"1,938,013,970,903","1,981,622,039,251"
3,282330,"2,946,235,271,626","2,596,378,756,682"
6,001460,"668,872,026,401","684,434,512,150"


In [ ]:
api_key = '4fb4acc1d37b1dae74040e64a9674e018ccfd7fe'
dart = OpenDartReader(api_key)
code, year = '311690', 2020

# 매출액
# 손익계산서
def sales_docu(code, year):
    sales = dart.finstate(code, year)
    # real_sales = sales[(sales['fs_nm']=='연결재무제표')&(sales['account_nm']=='매출액')].iloc[0, :] # 매출액 
    c_sales = sales[(sales['fs_nm']=='연결재무제표')&(sales['account_nm']=='매출액')]['thstrm_amount'].values # 매출액 # 문제1
    p_sales = sales[(sales['fs_nm']=='연결재무제표')&(sales['account_nm']=='매출액')]['frmtrm_amount'].values # t-1기 매출액 # 문제 2

    div_cash = dart.report(code, '배당', year) #(연결)현금배당성향(%)
    c_div = div_cash[div_cash['se']=='(연결)현금배당성향(%)']['thstrm'].values[0] # 순이익이 마이너스일 경우 0% # 배당금 / 당기순이익
    p_div = div_cash[div_cash['se']=='(연결)현금배당성향(%)']['frmtrm'].values[0]

    holder = dart.report(code, '최대주주', year) # 보통주 계 + 우선수 계 (%)
    ratio_stakeholder = holder[(holder['stock_knd']=='보통주')&(holder['nm']=='계')].values # 문제3

    exec = dart.report(code, '임원', year) #'사외이사' / 전체 임원 수   
    out_exec = exec[exec['ofcps'] == '사외이사'].shape[0]
    exec = exec.shape[0]

    employee = dart.report(code, '직원', year)['rgllbr_co'].to_list() # 총 직원 수
    
    output = {'sales': c_sales, 
              'pre_sales': p_sales, 
              'cash_div': c_div, 
              'pre_cash_civ': p_div,
              'stakeholder' : ratio_stakeholder,
              'num_outexecutives': out_exec,
              'num_executives': exec,
              'num_employee': employee}
    
    return output
# for 
data = []
good_index = []
bad_index = []

for index, code in enumerate(df1['code'].to_list()):
    try:
        data.append(sales_docu(code, year))
        print(f'{index}, Good!')
        good_index.append(index)
    except:
        print(f'{index}, bad...')
        bad_index.append(index)

In [26]:
print(len(good_index), len(bad_index)) # 

succees_sales_docu = df1.iloc[good_index, 0]
fail_sales_docu = df1.iloc[bad_index, 0]

691 16


In [25]:
with open ('s_docu.txt', 'w') as f:
    for x in df1.iloc[good_index, 0].values:
        f.write(f'{x}\n')
    f.close()
    
with open ('f_docu.txt', 'w') as f:
    for x in df1.iloc[bad_index, 0].values:
        f.write(f'{x}\n')
    f.close()

In [14]:
def html_extract(self, code, year):
